## Option类型，避免null的使用
我们来讨论3 种类型，即Option、Some 和None，它们可以表示“有值”或者“没有值”。

大部分语言都有一个特殊的关键字或类的特殊实例，用于在引用变量没有指向任何对象时，表示“无”。在Java 中，是null 关键字，但Java 中没有某个实例或类型。因此，对它调用任何方法都是非法的。但是语言设计者对此感到非常迷惑：为什么要在程序员期望返回对象时返回一个关键字呢？

当然，真正的问题在于，null 是很多bug 的来源。null 表示的真正含义是在给定的情形下没有任何值。如何变量不等于null，它是有值的。为什么不在类型系统中显式地将这种情况表达出来，并通过类型检查来避免空指针异常呢？

Option 允许我们显式表示这种情况，而不需要用null 这种“骇客”技巧。作为一个抽象类，Option 却有两个具体的子类Some 和None 。Some 用于表示有值，None 用于表示没有值。

为了让所有东西都是对象的目标更加一致，也为了遵循函数式编程的习惯，scala鼓励你在变量和函数返回值可能不会引用任何值的时候使用Option类型。在没有值的时候，使用None，它是Option的一个子类；在有值的时候，就用Some来包含这个值，它也是Option的一个子类。

在以下实例中你会看到Option、Some 和None 的运用。我们创建了一个美国州与州首府的映射表：

In [2]:
val stateCapitals = Map(
  "Alabama" -> "Montgomery",
  "Alaska"  -> "Juneau",
  // ...
  "Wyoming" -> "Cheyenne")
    
println( "Get the capitals wrapped in Options:" )
println( "Alabama: " + stateCapitals.get("Alabama") )
println( "Wyoming: " + stateCapitals.get("Wyoming") )
println( "Unknown: " + stateCapitals.get("Unknown") )

println()

println( "Get the capitals themselves out of the Options:" )
println( "Alabama: " + stateCapitals.get("Alabama").get )
println( "Wyoming: " + stateCapitals.get("Wyoming").getOrElse("Oops!") )
println( "Unknown: " + stateCapitals.get("Unknown").getOrElse("Oops2!") )

Get the capitals wrapped in Options:
Alabama: Some(Montgomery)
Wyoming: Some(Cheyenne)
Unknown: None

Get the capitals themselves out of the Options:
Alabama: Montgomery
Wyoming: Cheyenne
Unknown: Oops2!


stateCapitals: Map[String, String] = Map("Alabama" -> "Montgomery", "Alaska" -> "Juneau", "Wyoming" -> "Cheyenne")

Map.get 返回了Option，这明显告诉读者映射表中有可能找不到指定的key。映射表在这种情况下会返回None，而大部分编程语言会返回null（或其他等价的值）。的确，从经验来看你推断这种情况下这些语言中可能出现null，但Option 将这种情况显式地体现在函数签名中，使其更具自解释性。

另外，多亏Scala 的静态类型性质，你可以避免“忘记”返回的是Option，从而调用Option 里的值（如果有值的话）来启动方法。在Java 中，当方法返回一个值时，在调用该值的方法前很容易忘记检查它是否为null。但是，**Scala 的方法返回Option，编译器的类型检查便强制要求你先从Option 中提取值，再对它调用方法。这一机制“提醒”你去检查Option 是否等于None。所以，Option 的使用强烈鼓励更具弹性的编程习惯。**

Option.get 方法有些危险，如果Option 是一个Some，Some.get 则会返回其中的值。然而，如果Option 事实上是一个None，None.get 就会抛出一个NoSuchElementException 异常。

get 的替代选项—— 一个更安全的方法getOrElse。getOrElse 方法会在Option 为Some 时返回其中的值，而在Option 为None 时返回传递给它的参数中的值。换言之，getOrElse 的参数起到了默认值的作用。所以，getOrElse 是两个方法中更具防御性的，它避免了潜在的异常。

## Option当成是一个集合
Option的一个非常惯用的做法是，把Option当成是一个集合看待，集合的大小是1或者是0。当你的Option里面有东西的时候，它的长度是1（也就是 Some），而当你的Option里没有东西的时候，它的长度是0（也就是 None）。

**可以把map，filter，foreach等操作用在Option上。**

```scala
val name: Option[String] = request getParameter "name"
val upper = name map { _.trim } filter { _.length != 0 } map { _.toUpperCase }
println(upper getOrElse "")
```

这是Option官方给的一段代码，意思是从request中取出一个name值（但是name值可能是不存在的），然后对这个值去空格，去掉空值，转换为大写等一系列操作，最后拿到结果。

可以看到，这里没有任何if (name == null ) 或者 if (name.length == 0) 等这样的if语句，全是使用了scala推荐的函数式编程的写法，这种写法看起来更加scala，更简洁流畅，推荐这样写。

## 使用模式匹配
使用模式匹配也能达到避免使用if的效果，下面再贴一个官方的代码
```scala
val nameMaybe = request getParameter "name"
nameMaybe match {
  case Some(name) =>
    println(name.trim.toUppercase)
  case None =>
    println("No name value")
}
```
有值时，走Some(name) 后的逻辑，没有值的时候，走None 的逻辑，看起来也是非常明确易懂

## for循环中Option的模式匹配
如果我们把Option当成一般的List来用，并且用一个for循环来走访这个Option的时候，如果Option是None，那这个for循环里的程序代码自然不会执行，于是我们就达到了不用检查Option是否为None这件事。

**假如你需要对Option 对象进行检测，当它是Some 对象时执行一些操作，而当它是None 对象时则不进行任何操作，那么你可以使用for 推导式，这也
是Scala 的一个广泛应用的常见用法。**
```scala
  import scala.io.Source
  import scala.util.control.NonFatal

  def countLines(fileName: String) = {   
    var source: Option[Source] = None
    try {
      source = Some(Source.fromFile(fileName))
      val size = source.get.getLines.size
      println(s"file $fileName has $size lines")
    } catch {
      case NonFatal(ex) => println(s"Non fatal exception! $ex")
    } finally {
      //关闭资源时就可以使用，避免判断null
      for (s <- source) {
        println(s"Closing $fileName...")
        s.close
      }
    }
  }
```

In [11]:
//x循环的Some中的元素
for (x <- Some("Doberman")){
    println(x + ": " + x.getClass)
}

Doberman: class java.lang.String


In [12]:
//会直接跳过
for (x <- None){
    println(x)
}

看看下面的代码：

In [4]:
val dogBreeds = List("Doberman", "Yorkshire Terrier", "Dachshund")
for {
  breed <- dogBreeds
  upcasedBreed = breed.toUpperCase()
} println(upcasedBreed)

DOBERMAN
YORKSHIRE TERRIER
DACHSHUND


dogBreeds: List[String] = List("Doberman", "Yorkshire Terrier", "Dachshund")

如果list中有null，怎么过滤呢：

In [5]:
val dogBreeds = List("Doberman", "Yorkshire Terrier", "Dachshund", null)
for {
  breed <- dogBreeds
  if breed != null
  upcasedBreed = breed.toUpperCase()
} println(upcasedBreed)

DOBERMAN
YORKSHIRE TERRIER
DACHSHUND


dogBreeds: List[String] = List("Doberman", "Yorkshire Terrier", "Dachshund", null)

如果你想到了Option，那就可以用在这个示例中。正如我们之前讨论的那样，Option 是null 更好的替代方案，Option 是一类特殊形式的集合，它只包含0 个或1 个元素，意识到这一点对你会有帮助。我们也可以理解下面代码：

In [17]:
val dogBreeds = List(Some("Doberman"), Some("Yorkshire Terrier"), Some("Dachshund"), None)
println("first pass:")
for {
  breedOption <- dogBreeds
  breed <- breedOption
  upcasedBreed = breed.toUpperCase()
} println(upcasedBreed)

println("\nsecond pass:")
for {
  breedOption <- dogBreeds
  upcasedBreed = breedOption.getOrElse("00000").toUpperCase()
} println(upcasedBreed)

println("\nthird pass:")
for {
  breedOption <- dogBreeds
  if breedOption != None
  upcasedBreed = breedOption.get.toUpperCase()
} println(upcasedBreed)

println("\nfourth pass:")
for {
  Some(breed) <- dogBreeds
  upcasedBreed = breed.toUpperCase()
} println(upcasedBreed)

first pass:
DOBERMAN
YORKSHIRE TERRIER
DACHSHUND

second pass:
DOBERMAN
YORKSHIRE TERRIER
DACHSHUND
00000

third pass:
DOBERMAN
YORKSHIRE TERRIER
DACHSHUND

fourth pass:
DOBERMAN
YORKSHIRE TERRIER
DACHSHUND


dogBreeds: List[Option[String]] = List(Some("Doberman"), Some("Yorkshire Terrier"), Some("Dachshund"), None)